# **뉴스 데이터 - 부동산 뉴스기사 키워드 추출을 통한 연도별 이슈 분석**

- 필요한 라이브러리 다운로드

In [1]:
#한글 폰트 다운로드
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 17 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (8,518 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 120901 files and dire

In [2]:
#크롤링 user_agent 생성 라이브러리 다운로드
!pip install user_agent

  Preparing metadata (setup.py) ... done
  Created wheel for user_agent: filename=user_agent-0.1.10-py3-none-any.whl size=18966 sha256=4fe3d85a1bde4ff5b8a5aee331ecd22808715bb73e10c558a191ba1e704c0b7f
  Stored in directory: /root/.cache/pip/wheels/69/29/26/1956a891a058037774285ee79ab5c3ecf034dba50a4198fedd
Successfully built user_agent


In [3]:
#한글 형태소 분석기 다운로드
!pip install konlpy
!pip install mecab-python
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 29.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.7/581.7 kB 4.4 MB/s eta 0:00:00
  Created wheel for mecab-python: filename=mecab_python-1.0.0-py3-none-any.whl size=1231 sha256=f6dfed5da336a84f6fb1636f0ab3119632ddba42e9a670b910e94e878864f24b
  Stored in directory: /root/.cache/pip/wheels/ec/41/c8/ce8fb469be2547ce596c4000613806505dc54c296aae45da71
Successfully built mecab-python
Install mecab-ko
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1381k  100 1381k    0     0  2063k      0 --:--:-- --:--:-- --:--:-- 26.1M
mecab-0.996-ko-0.9.2/
mecab-0.996-ko-0.9.2/example/
mecab-0.996-ko-0.9.2/example/ex

## **데이터 수집**

- 2018년도부터 2023년 8월까지 '부동산' 키워드 뉴스 기사 크롤링

In [3]:
from user_agent import generate_user_agent, generate_navigator
user_agent = generate_user_agent()
user_agent

'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:47.0) Gecko/20100101 Firefox/47.0'

In [4]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import random
import time
from tqdm.notebook import tqdm

In [8]:
first_day = pd.date_range('2018-01-01', '2023-08-31', freq='MS').strftime('%Y.%m.%d')
last_day = pd.date_range('2018-01-01', '2023-08-31', freq='M').strftime('%Y.%m.%d')
date_list = list(zip(first_day, last_day))

In [91]:
keyword = '부동산'

data = pd.DataFrame()
for dt in tqdm(date_list):
    dt1 = [dt[0].replace('.',''), dt[1].replace('.','')]

    for num in range(1, 101, 10):
        url = f'https://search.naver.com/search.naver?where=news&query={keyword}&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds={dt[0]}&de={dt[1]}&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom{dt1[0]}to{dt1[1]}&is_sug_officeid=0&office_category=0&service_area=1&start={num}'

        user_agent = generate_user_agent()
        headers = {'User-Agent':user_agent}

        res = requests.get(url, headers=headers)
        time.sleep(random.random())
        soup = bs(res.text, 'html.parser')

        title = [i.text for i in soup.find_all('a', class_='news_tit')]
        link = [i['href'] for i in soup.find_all('a', class_='news_tit')]
        press = [i.text for i in soup.find_all('a', class_='info press')]

        temp = pd.DataFrame({'title':title, 'link':link, 'press':press, 'date_ym':dt1[0][:6]})

        if len(title) < 1 : #크롤링이 아무것도 되지 않았으면 루프를 종료, 시간 텀을 가진 뒤 다시 크롤링
            break

        data = pd.concat([data, temp])

  0%|          | 0/68 [00:00<?, ?it/s]

In [92]:
data.to_excel('/content/drive/MyDrive/강의자료_황수현_Python/Part3) 파이썬 데이터 분석 프로젝트/data/실습7: 직접 크롤링하기 - 부동산 뉴스기사 키워드 추출을 통한 연도별 이슈 분석/news_real_estate.xlsx')

## **질문 만들기**

- 연도별로 부동산 전망에 대한 뉴스 키워드는 어떻게 변화했는가?
    - 워드클라우드로 시각화

## **데이터 전처리**

- 필요 없는 컬럼 제거 및 데이터 타입 변경

In [104]:
data = pd.read_excel('/content/drive/MyDrive/강의자료_황수현_Python/Part3) 파이썬 데이터 분석 프로젝트/data/실습7: 직접 크롤링하기 - 부동산 뉴스기사 키워드 추출을 통한 연도별 이슈 분석/news_real_estate.xlsx')

In [105]:
data.head()

,Unnamed: 0,title,link,press,date_ym
0,0,부자들 60% “부동산 규제에도 집 안 판다”,http://www.hankookilbo.com/v/26932a67d1b242f99...,한국일보언론사 선정,201801
1,1,"""파느니 물려준다"" 지난해 부동산 증여 28만2천건 역대 최대",http://app.yonhapnews.co.kr/YNA/Basic/SNS/r.as...,연합뉴스,201801
2,2,부동산 중개사이트 허위매물 제재한다,http://news.heraldcorp.com/view.php?ud=2018013...,헤럴드경제,201801
3,3,"김현미 장관 ""부동산 침체 지역, '위축지역' 지정 검토""",http://www.newsis.com/view/?id=NISX20180130_00...,뉴시스언론사 선정,201801
4,4,"KB운용, 글로벌 대체투자 강화…해외부동산 전담본부 신설",http://news.mt.co.kr/mtview.php?no=20180131085...,머니투데이,201801


In [106]:
data = data[['title','date_ym']]

In [107]:
data['date_ym'] = data['date_ym'].astype(str)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6800 entries, 0 to 6799
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    6800 non-null   object
 1   date_ym  6800 non-null   object
dtypes: object(2)
memory usage: 106.4+ KB


In [108]:
data.head()

,title,date_ym
0,부자들 60% “부동산 규제에도 집 안 판다”,201801
1,"""파느니 물려준다"" 지난해 부동산 증여 28만2천건 역대 최대",201801
2,부동산 중개사이트 허위매물 제재한다,201801
3,"김현미 장관 ""부동산 침체 지역, '위축지역' 지정 검토""",201801
4,"KB운용, 글로벌 대체투자 강화…해외부동산 전담본부 신설",201801


- 형태소 분석

In [109]:
from konlpy.tag import Mecab

mecab = Mecab()

In [122]:
data['NNG'] = data['title'].apply(lambda x: [i[0] for i in mecab.pos(x) if i[1] in ("NNG")])

In [123]:
data['year'] = data['date_ym'].apply(lambda x: int(x[:4]))
data.head()

,title,date_ym,year,NNG
0,부자들 60% “부동산 규제에도 집 안 판다”,201801,2018,"[부자, 부동산, 규제, 집]"
1,"""파느니 물려준다"" 지난해 부동산 증여 28만2천건 역대 최대",201801,2018,"[지난해, 부동산, 증여, 역대, 최대]"
2,부동산 중개사이트 허위매물 제재한다,201801,2018,"[부동산, 중개, 사이트, 허위, 매물, 제재]"
3,"김현미 장관 ""부동산 침체 지역, '위축지역' 지정 검토""",201801,2018,"[장관, 부동산, 침체, 지역, 위축, 지역, 지정, 검토]"
4,"KB운용, 글로벌 대체투자 강화…해외부동산 전담본부 신설",201801,2018,"[운용, 글로벌, 대체, 투자, 강화, 해외, 부동산, 전담, 본부, 신설]"


In [124]:
from collections import Counter

In [131]:
nng_2018 = [j for i in data.query('year == 2018')['NNG'] for j in i if j not in ['부동산','전망','시장','집값','투자'] and len(j) > 1]
nng_2019 = [j for i in data.query('year == 2019')['NNG'] for j in i if j not in ['부동산','전망','시장','집값','투자'] and len(j) > 1]
nng_2020 = [j for i in data.query('year == 2020')['NNG'] for j in i if j not in ['부동산','전망','시장','집값','투자'] and len(j) > 1]
nng_2021 = [j for i in data.query('year == 2021')['NNG'] for j in i if j not in ['부동산','전망','시장','집값','투자'] and len(j) > 1]
nng_2022 = [j for i in data.query('year == 2022')['NNG'] for j in i if j not in ['부동산','전망','시장','집값','투자'] and len(j) > 1]
nng_2023 = [j for i in data.query('year == 2023')['NNG'] for j in i if j not in ['부동산','전망','시장','집값','투자'] and len(j) > 1]

nng_2018_dict = dict(Counter(nng_2018).most_common(100))
nng_2019_dict = dict(Counter(nng_2019).most_common(100))
nng_2020_dict = dict(Counter(nng_2020).most_common(100))
nng_2021_dict = dict(Counter(nng_2021).most_common(100))
nng_2022_dict = dict(Counter(nng_2022).most_common(100))
nng_2023_dict = dict(Counter(nng_2023).most_common(100))

## **분석**

In [102]:
import matplotlib.pyplot as plt
plt.rc('font', family='NanumGothic')

from wordcloud import WordCloud

path = '/user/share/fonts/truetype/nanum/NanumGothic.ttf'
wc = WordCloud(font_path = path,
               background_color='white',
               width=1000,
               height=1000,
               max_font_size=300)

dict_list = [nng_2018_dict, nng_2019_dict, nng_2020_dict, nng_2021_dict, nng_2022_dict, nng_2023_dict]
title_list = [i for i in range(2018, 2024)]

fig = plt.figure(figsize=(25,20))
for i in range(len(dict_list)):
    wc.generate_from_frequencies(dict_list[i]) #워드클라우드 생성
    ax = fig.add_subplot(2,3,i+1)
    ax.imshow(wc, interpolation='bilinear')
    ax.set_xlabel(f'{title_list[i]}') #그래프 제목 출력
    ax.set_xticks([]), ax.set_yticks([]) #x축, y축을 없앰
    plt.imshow(wc, interpolation='bilinear')
fig.suptitle('부동산 뉴스 제목 키워드')
fig.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.